In [1]:
import pandas as pd 
import datetime
data = pd.read_csv('newData.csv')
user_id_list = data['user_id'].unique()
print(user_id_list, len(user_id_list))

[    0     2     3 ... 34471 34483 34484] 10096


In [2]:
new_data = {'user_id': user_id_list, 'car_ownership': 0}
df = pd.DataFrame(new_data)
df

,user_id,car_ownership
0,0,0
1,2,0
2,3,0
3,8,0
4,10,0
...,...,...
10091,34467,0
10092,34468,0
10093,34471,0
10094,34483,0


In [3]:
tripleg = pd.read_csv('triplegs.csv')
stop_points = pd.read_csv('staypoints.csv')

In [4]:
stop_points['started_at'] = pd.to_datetime(stop_points['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])

In [5]:
ID_list=stop_points['user_id'].unique()
len(ID_list)

10096

In [6]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
import trackintel as ti
import numpy as np
tpls = ti.io.file.read_triplegs_csv('trip_5.csv', columns={'trip_started_at':'started_at', 'trip_finished_at':'finished_at', 'trip':'geom'}, crs='EPSG:4326', index_col=0)

In [7]:
# Predict the transport mode of triplegs.
"""simple-coarse method includes {'slow_mobility', 'motorized_mobility', 'fast_mobility'}. 
In the default classification, slow_mobility (<15 km/h) includes transport modes such as walking or cycling, 
motorized_mobility (<100 km/h) modes such as car or train, 
and fast_mobility (>100 km/h) modes such as high-speed rail or airplanes."""
categories = {
    15/3.6: 'slow_mobility',  # walking or cycling
    193/3.6: 'motorized_mobility',  # car
    np.inf: 'fast_mobility'  # high-speed rail or airplane
}
mode_tpls  = tpls.as_triplegs.predict_transport_mode(categories = categories)
mode_tpls

/usr/local/lib/python3.9/site-packages/pygeos/io.py:89: UserWarning: The shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the PyGEOS GEOS version (3.10.4-CAPI-1.16.2). Conversions between both will be slow
  warnings.warn(


,user_id,tripleg_ID,started_at,finished_at,geom,mode
0,10,758,2022-10-07 22:00:05-04:00,2022-10-07 22:00:34-04:00,"LINESTRING (-81.65050 30.47010, -81.63820 30.4...",motorized_mobility
1,10,764,2022-10-08 13:45:17-04:00,2022-10-08 13:45:27-04:00,"LINESTRING (-81.63415 30.47833, -81.63416 30.4...",slow_mobility
2,10,770,2022-10-09 13:00:02-04:00,2022-10-09 13:00:51-04:00,"LINESTRING (-81.62109 30.47463, -81.62109 30.4...",slow_mobility
3,10,797,2022-10-11 22:00:32-04:00,2022-10-11 22:00:46-04:00,"LINESTRING (-81.65050 30.47010, -81.65050 30.4...",slow_mobility
4,14,930,2022-09-11 16:42:15-04:00,2022-09-11 16:42:40-04:00,"LINESTRING (-82.63589 29.44469, -82.63659 29.4...",motorized_mobility
...,...,...,...,...,...,...
40277,34471,837931,2022-10-25 20:03:34-04:00,2022-10-25 20:10:21-04:00,"LINESTRING (-82.80514 30.31518, -82.80514 30.3...",motorized_mobility
40278,34471,837932,2022-10-25 20:25:42-04:00,2022-10-25 20:27:53-04:00,"LINESTRING (-82.57936 29.97980, -82.56936 29.9...",motorized_mobility
40279,34471,837934,2022-10-30 01:52:42-04:00,2022-10-30 01:54:53-04:00,"LINESTRING (-82.78531 30.29825, -82.81831 30.3...",motorized_mobility
40280,34471,837935,2022-10-30 02:08:36-04:00,2022-10-30 02:08:46-04:00,"LINESTRING (-82.98503 30.48014, -82.98502 30.4...",slow_mobility


In [8]:
mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=mode_tpls, metric='count', per_user=True)
mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
10,0,1,3
14,0,1,0
22,2,70,32
30,0,0,1
32,0,0,1
...,...,...,...
34391,0,1,0
34405,0,5,0
34406,0,3,0


In [9]:
index = mode_split.index.values
for id in index:
    motor_count = mode_split.loc[id, 'fast_mobility'] + mode_split.loc[id, 'motorized_mobility'] + mode_split.loc[id, 'slow_mobility']
    if mode_split.loc[id, 'motorized_mobility'] / motor_count > 0.5:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,2,0
2,3,0
3,8,0
4,10,0
...,...,...
10091,34467,0
10092,34468,0
10093,34471,0
10094,34483,0


In [10]:
print(len(df[df['car_ownership'] == 1]))

2161


In [11]:
home_location = pd.read_csv('home_location.csv', index_col=0)
home_location

,ID,LAT,LON,LAT-4326,LON-4326
0,0,3550540.0,-9092080.0,30.364891,-81.675544
1,2,3458860.0,-9093020.0,29.651722,-81.683988
2,3,3456360.0,-9163720.0,29.632203,-82.319097
3,8,3510020.0,-9105740.0,30.050323,-81.798254
4,10,3565080.0,-9087560.0,30.477524,-81.634940
...,...,...,...,...,...
9362,34077,3462400.0,-9091920.0,29.679354,-81.674107
9363,34249,3373140.0,-9053960.0,28.980309,-81.333106
9364,34319,3535180.0,-9080400.0,30.245766,-81.570621
9365,34373,3541500.0,-9103240.0,30.294798,-81.775796


In [12]:
from geopy.distance import geodesic
from shapely.wkt import loads
from tqdm import tqdm
data_to_append_list = []
home_id_list = home_location['ID'].unique()
for id in tqdm(home_id_list, total = len(home_id_list)):
    # print(home_location.loc[home_location['ID'] == id, 'LAT-4326'].values[0])
    p1 = (home_location.loc[home_location['ID'] == id, "LAT-4326"].values[0], home_location.loc[home_location['ID'] == id, "LON-4326"].values[0])
    # print(p1)
    for sp in stop_points[stop_points['user_id'] == id].itertuples():
        p = loads(sp[5])
        lat = p.y
        lon = p.x
        p2 = (lat, lon)
        # print(p2)
        if geodesic(p1, p2).meters < 100:
            data_to_append_list.append(sp)
new_staypoints = pd.DataFrame.from_records(data_to_append_list)
new_staypoints


100%|██████████| 9367/9367 [01:09<00:00, 135.39it/s]


,0,1,2,3,4,5
0,2,2,2,2022-09-06 22:19:08-04:00,2022-09-06 22:22:07-04:00,POINT (-81.6840639999999922 29.6517550000000014)
1,4,4,2,2022-09-06 22:31:37-04:00,2022-09-06 22:34:49-04:00,POINT (-81.6840650000000181 29.6517550000000014)
2,17,17,3,2022-09-05 19:40:35-04:00,2022-09-05 20:01:05-04:00,POINT (-82.3190549210000029 29.6321820860000074)
3,19,19,3,2022-09-07 19:39:17-04:00,2022-09-07 20:02:10-04:00,POINT (-82.3190553007142825 29.6321777757142861)
4,21,21,3,2022-09-08 19:40:41-04:00,2022-09-08 20:01:10-04:00,POINT (-82.3190579438461469 29.6321850161538478)
...,...,...,...,...,...,...
46550,310770,310770,33722,2022-10-22 13:59:34-04:00,2022-10-22 14:11:06-04:00,POINT (-82.3622717500000050 29.5906182000000015)
46551,310781,310781,33744,2022-10-24 10:50:50-04:00,2022-10-24 10:57:41-04:00,POINT (-82.3449399999999940 29.5589399999999998)
46552,310786,310786,33744,2022-10-26 10:50:02-04:00,2022-10-26 10:54:05-04:00,POINT (-82.3449470049999945 29.5589244700000009)
46553,310793,310793,33744,2022-10-29 10:50:38-04:00,2022-10-29 10:59:48-04:00,POINT (-82.3449399999999940 29.5589300000000001)


In [13]:
drop_cols = [0,1]
new_staypoint = new_staypoints.drop(new_staypoints.columns[drop_cols], axis=1)
new_staypoint = new_staypoint.rename(columns={2:'user_id', 3: 'started_at', 4: 'finished_at', 5: 'geom'})
new_staypoint


,user_id,started_at,finished_at,geom
0,2,2022-09-06 22:19:08-04:00,2022-09-06 22:22:07-04:00,POINT (-81.6840639999999922 29.6517550000000014)
1,2,2022-09-06 22:31:37-04:00,2022-09-06 22:34:49-04:00,POINT (-81.6840650000000181 29.6517550000000014)
2,3,2022-09-05 19:40:35-04:00,2022-09-05 20:01:05-04:00,POINT (-82.3190549210000029 29.6321820860000074)
3,3,2022-09-07 19:39:17-04:00,2022-09-07 20:02:10-04:00,POINT (-82.3190553007142825 29.6321777757142861)
4,3,2022-09-08 19:40:41-04:00,2022-09-08 20:01:10-04:00,POINT (-82.3190579438461469 29.6321850161538478)
...,...,...,...,...
46550,33722,2022-10-22 13:59:34-04:00,2022-10-22 14:11:06-04:00,POINT (-82.3622717500000050 29.5906182000000015)
46551,33744,2022-10-24 10:50:50-04:00,2022-10-24 10:57:41-04:00,POINT (-82.3449399999999940 29.5589399999999998)
46552,33744,2022-10-26 10:50:02-04:00,2022-10-26 10:54:05-04:00,POINT (-82.3449470049999945 29.5589244700000009)
46553,33744,2022-10-29 10:50:38-04:00,2022-10-29 10:59:48-04:00,POINT (-82.3449399999999940 29.5589300000000001)


In [14]:
time_window = datetime.timedelta(minutes=5)

miss = 0
check_len = 0
data_to_append_list = []
for id in ID_list:
    related_stop_by_user = new_staypoint[new_staypoint['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for stop_related_row in related_stop_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_related_row[2] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_related_row[2])
        ]
        #print(len(tripled_found))
        if len(tripled_found) == 0: continue #did not found related trip
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        data_to_append = {'user_id': id, 'trip_started_at': tripled_found.iloc[0]['started_at'],
                'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                'trip':tripled_found.iloc[0]['geom']}
        
        data_to_append_list.append(data_to_append)
        

new_trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'trip_started_at', 'trip_finished_at', 'trip'])
new_trip_df

,user_id,trip_started_at,trip_finished_at,trip
0,22,2022-09-09 20:33:27-04:00,2022-09-09 20:35:51-04:00,LINESTRING (-81.4925243999999935 30.3372013000...
1,22,2022-09-10 13:13:33-04:00,2022-09-10 13:13:44-04:00,LINESTRING (-81.4755399999999952 30.3195799999...
2,22,2022-09-10 13:49:34-04:00,2022-09-10 13:50:28-04:00,LINESTRING (-81.4898749999999978 30.3275840000...
3,22,2022-09-13 13:49:37-04:00,2022-09-13 13:49:47-04:00,LINESTRING (-81.4898749999999978 30.3275840000...
4,22,2022-09-15 22:42:23-04:00,2022-09-15 22:43:24-04:00,LINESTRING (-81.4953000000000003 30.3301900000...
...,...,...,...,...
3376,33446,2022-10-28 21:06:11-04:00,2022-10-28 21:07:46-04:00,LINESTRING (-82.2479710000000068 29.2432419999...
3377,33747,2022-10-25 01:20:59-04:00,2022-10-25 01:22:52-04:00,LINESTRING (-81.7429779999999937 30.3809520000...
3378,33747,2022-10-25 02:23:18-04:00,2022-10-25 02:25:12-04:00,LINESTRING (-81.7518210399999958 30.3788065899...
3379,33747,2022-10-25 03:19:35-04:00,2022-10-25 03:20:42-04:00,LINESTRING (-81.7520785300000057 30.3786778499...


In [15]:
new_trip_df.to_csv('home_related_trip.csv')

In [16]:
new_tpls = ti.io.file.read_triplegs_csv('home_related_trip.csv', columns={'trip_started_at':'started_at', 'trip_finished_at':'finished_at', 'trip':'geom'}, crs='EPSG:4326', index_col=0)
new_mode_tpls  = new_tpls.as_triplegs.predict_transport_mode(categories = categories)
new_mode_tpls

,user_id,started_at,finished_at,geom,mode
0,22,2022-09-09 20:33:27-04:00,2022-09-09 20:35:51-04:00,"LINESTRING (-81.49252 30.33720, -81.49291 30.3...",slow_mobility
1,22,2022-09-10 13:13:33-04:00,2022-09-10 13:13:44-04:00,"LINESTRING (-81.47554 30.31958, -81.47554 30.3...",slow_mobility
2,22,2022-09-10 13:49:34-04:00,2022-09-10 13:50:28-04:00,"LINESTRING (-81.48987 30.32758, -81.49529 30.3...",motorized_mobility
3,22,2022-09-13 13:49:37-04:00,2022-09-13 13:49:47-04:00,"LINESTRING (-81.48987 30.32758, -81.48987 30.3...",slow_mobility
4,22,2022-09-15 22:42:23-04:00,2022-09-15 22:43:24-04:00,"LINESTRING (-81.49530 30.33019, -81.49530 30.3...",motorized_mobility
...,...,...,...,...,...
3376,33446,2022-10-28 21:06:11-04:00,2022-10-28 21:07:46-04:00,"LINESTRING (-82.24797 29.24324, -82.25425 29.2...",motorized_mobility
3377,33747,2022-10-25 01:20:59-04:00,2022-10-25 01:22:52-04:00,"LINESTRING (-81.74298 30.38095, -81.74297 30.3...",motorized_mobility
3378,33747,2022-10-25 02:23:18-04:00,2022-10-25 02:25:12-04:00,"LINESTRING (-81.75182 30.37881, -81.75206 30.3...",slow_mobility
3379,33747,2022-10-25 03:19:35-04:00,2022-10-25 03:20:42-04:00,"LINESTRING (-81.75208 30.37868, -81.75208 30.3...",slow_mobility


In [17]:
new_mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=new_mode_tpls, metric='count', per_user=True)
new_mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
22,0,6,6
74,0,2,2
109,0,5,0
117,0,0,1
176,0,0,1
...,...,...,...
33252,0,0,1
33380,0,1,0
33446,0,1,0


In [18]:
index = new_mode_split.index.values
for id in index:
    motor_count = new_mode_split.loc[id, 'motorized_mobility']
    if motor_count > 0:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,2,0
2,3,0
3,8,0
4,10,0
...,...,...
10091,34467,0
10092,34468,0
10093,34471,0
10094,34483,0


In [19]:
print(len(df[df['car_ownership'] == 1]))

2328


In [20]:
all_tpls = ti.io.file.read_triplegs_csv('triplegs.csv', index_col=0)
all_mode_tpls  = all_tpls.as_triplegs.predict_transport_mode(categories = categories)
all_mode_tpls

/usr/local/lib/python3.9/site-packages/trackintel/geogr/distances.py:227: UserWarning: The CRS of your data is not defined.
  warnings.warn("The CRS of your data is not defined.")


,user_id,started_at,finished_at,geom,mode
id,,,,,
1,1,2022-09-06 13:11:03-04:00,2022-09-06 13:11:54-04:00,"LINESTRING (-82.07077 29.19301, -82.07077 29.1...",slow_mobility
5,1,2022-09-09 00:59:39-04:00,2022-09-09 00:59:49-04:00,"LINESTRING (-82.11661 29.19956, -82.11661 29.1...",slow_mobility
6,2,2022-09-06 09:15:08-04:00,2022-09-06 09:15:58-04:00,"LINESTRING (-81.67753 29.65504, -81.67751 29.6...",slow_mobility
7,2,2022-09-06 21:34:07-04:00,2022-09-06 21:34:57-04:00,"LINESTRING (-81.68406 29.65178, -81.68406 29.6...",slow_mobility
8,2,2022-09-07 00:05:48-04:00,2022-09-07 00:09:57-04:00,"LINESTRING (-81.65829 29.65349, -81.68407 29.6...",motorized_mobility
...,...,...,...,...,...
837979,34494,2022-10-27 01:06:08-04:00,2022-10-27 01:09:53-04:00,"LINESTRING (-81.61198 30.38249, -81.61198 30.3...",slow_mobility
837980,34496,2022-10-31 02:06:17-04:00,2022-10-31 02:07:50-04:00,"LINESTRING (-81.75724 30.33606, -81.75724 30.3...",slow_mobility
837981,34497,2022-10-26 08:25:34-04:00,2022-10-26 08:25:48-04:00,"LINESTRING (-82.09808 29.99144, -82.09809 29.9...",slow_mobility


In [21]:
all_mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=all_mode_tpls, metric='count', per_user=True)
all_mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
1,0,0,2
2,0,9,23
3,0,0,126
6,0,0,1
8,0,4,380
...,...,...,...
34484,0,0,1
34485,0,1,0
34494,0,0,28


In [22]:
index = all_mode_split.index.values
for id in index:
    motor_count = all_mode_split.loc[id, 'motorized_mobility']
    if motor_count > 0:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,2,1
2,3,0
3,8,1
4,10,1
...,...,...
10091,34467,0
10092,34468,0
10093,34471,1
10094,34483,0


In [23]:
home_locations = home_location.rename(columns={'ID':'user_id'})
home_locations
merged_data = pd.merge(df, home_locations[['user_id','LAT-4326','LON-4326']], on='user_id', how='left')
merged_data

,user_id,car_ownership,LAT-4326,LON-4326
0,0,0,30.364891,-81.675544
1,2,1,29.651722,-81.683988
2,3,0,29.632203,-82.319097
3,8,1,30.050323,-81.798254
4,10,1,30.477524,-81.634940
...,...,...,...,...
10091,34467,0,30.202919,-81.748487
10092,34468,0,29.291343,-82.516367
10093,34471,1,28.953272,-82.120031
10094,34483,0,30.000546,-81.370836


In [24]:
print(len(df[df['car_ownership'] == 1]))

5523


In [31]:
percentage = len(df[df['car_ownership'] == 1]) / df.shape[0]
percentage_string = "{:.0%}".format(percentage)
print(percentage_string)

55%


In [25]:
merged_data.to_csv('User.csv')